1. Read Mask Rcnn model
2. Read image in folder
3. 

In [1]:
import tensorflow as tf
import pathlib
import time
import cv2
from PIL import Image
import numpy as np
from object_detection.utils import ops as utils_ops
import matplotlib.pyplot as plt
import json
import codecs
import io
import base64
import os
%matplotlib inline

In [2]:
def load_model(model_dir):
    
    model = tf.saved_model.load(str(model_dir))
    return model

In [3]:
model_dir = r'D:\(Code)\Python\TF2_MASKRCNN_TUTORIAL\saved_model\saved_model'
masking_model = load_model(model_dir)

In [4]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path(r'D:\(Code)\Python\TF2_MASKRCNN_TUTORIAL\dataset\test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))

In [5]:
def encodeImageForJson(image):
    img_pil = Image.fromarray(image, mode='RGB')
    f = io.BytesIO()
    img_pil.save(f, format='PNG')
    data = f.getvalue()
    encData = codecs.encode(data, 'base64').decode()
    encData = encData.replace('\n', '')
    return encData

In [6]:
def img_arr_to_b64(img_arr):
    img_pil = Image.fromarray(img_arr)
    f = io.BytesIO()
    img_pil.save(f, format='PNG')
    img_bin = f.getvalue()
    if hasattr(base64, 'encodebytes'):
        img_b64 = base64.encodebytes(img_bin)
    else:
        img_b64 = base64.encodestring(img_bin)
    return img_b64

In [7]:
def run_inference_and_get_labelme_json(model, image_np, image_path, img_width, img_height):    
    print(image_path)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
        
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    detections = model(input_tensor)

    # Handle models with masks:
    if "detection_masks" in detections:
        # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(detections["detection_masks"][0], detections["detection_boxes"][0],image_np.shape[0], image_np.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,tf.uint8)
        detections["detection_masks_reframed"] = detection_masks_reframed.numpy()

    scores = np.asarray(detections["detection_scores"][0])    
    mask = np.asarray(detections["detection_masks_reframed"])
        

    ls = []
 
    ## Each image should be have their own mask and json
    for i, img in enumerate(mask):
        if(scores[i] > 0.5):        
            shape_dictionary = {"label" : "aruco",
                        "points" : [],
                        "group_id" : None,
                        "shape_type": "polygon",
                        "flags":{}}  
                
            img_mask = np.empty(shape=(mask.shape[1], mask.shape[2],3),dtype='uint8')
            
            ret, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY)
            countours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)                      
            epsilon = 0.1 * cv2.arcLength(countours[0], True)
            approx = cv2.approxPolyDP(countours[0], epsilon, True)   
            approx_new = np.reshape(approx, (4,2))            
            shape_dictionary["points"] = approx_new.tolist()
            #cv2.drawContours(img_mask, approx, -1, (0, 255, 0), 3)
            #display(Image.fromarray(img_mask))           
            ls.append(shape_dictionary) 
        
    dictionary = {
        "version": "5.0.1",
        "flags": {} ,
        "shapes": [],
        "imagePath" : "",
        "imageData": "",
        "imageHeight": "",
        "imageWidth": ""
    }
            
    dictionary["shapes"] = ls
    dictionary["imageData"] = encodeImageForJson(image_np)   
    
    dictionary["imagePath"] = image_path
    dictionary["imageHeight"] = img_height
    dictionary["imageWidth"] = img_width
    
    with open(f"{os.path.splitext(image_path)[0]}.json", "w") as outfile:
        json.dump(dictionary, outfile, indent=2)
            
    

In [8]:
def show_inference(model, image_path):
    
    img = Image.open(image_path)
    ori_width = img.width
    ori_height = img.height
    

    # Load image
    image_np = np.array(img)
    
    # Actual detection.
    img = run_inference_and_get_labelme_json(model, image_np, os.path.basename(str(image_path)), str(ori_width), str(ori_height))

    #display(img)


In [9]:
for image_path in TEST_IMAGE_PATHS:
    t1 = time.time()
    show_inference(masking_model, image_path)
    print(time.time() - t1)

WIN_20220711_22_37_23_Pro.jpg
15.220494747161865
